<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados.</font>
<br>
<font size='1'> Modificado en 2018-1, 2018-2 por Equipo Docente IIC2233</font>
</p>

## Iterar sobre estructuras de datos 

En muchas ocasiones implementaremos estructuras de datos en que resulta natural la noción de que pueden ser recorridas, este es el caso para estructuras que hemos visto como listas, tuplas, sets y diccionarios. Nos gustaría que, cuando hagamos nuestras propias estructuras, también puedan ser recorridas utilizando `for`, como lo haríamos con una lista. Para esto hay que entender dos conceptos claves: iterable e iterador.

Un **iterable** es cualquier objeto sobre el cual se puede iterar. Por lo mismo, cualquier iterable podría aparecer al lado derecho de un _for loop_ (`for i in iterable:`). Ejemplos de estructuras _built-ins_ que son iterables son los _sets_, listas, diccionarios y _deques_.

Un iterable implementa el método **`__iter__()`**. Se puede iterar todas las veces que uno quiera sobre un iterable, como en el caso de las listas por ejemplo. No es necesario que este objeto se pueda indexar. Por ejemplo los _sets_ no se indexan, pero si podemos iterar sobre ellos.

In [1]:
conjunto = {1, 3, 4, 6}

for i in conjunto:
    print(i)

1
3
4
6


Un **iterador** es un objeto que itera sobre un iterable, y es el objeto retornado por el método `__iter__()`. Este objeto iterador implementa el método `__next__()`, que nos retorna uno a uno los elementos de la estructura cada vez que se invoca a esta función. 

Además, si no quedan objetos por recorrer el iterador **debe** levantar una excepción de tipo `StopIteration`. 

In [2]:
conjunto = {1, 3, 4, 6}
iterador = iter(conjunto)  # Notar que es lo mismo que hacer conjunto.__iter__()

print(next(iterador))  # Notar que es lo mismo que iterador.__next__()
print(next(iterador))
print(next(iterador))

1
3
4


Veremos que si al iterador le pedimos más elementos de los que tiene la estructura, levantará una excepción:

In [3]:
print(next(iterador))
print(next(iterador))

6


StopIteration: 

**Recuerden** 

> Un iterable debe tener el método `__iter__` implementado, y debe retornar **siempre** un iterador. Por su parte, un iterador es un objeto que tiene el método `__next__` implementado, es decir puedo hacer `next(objeto)` y esto retornará un **valor**.


### Forma básica de hacer una estructura iterable

Ahora veremos la forma más básica de crear un iterable, que es creando una clase iteradora. Hay más formas de hacer una estructura iterable sin crear una nueva clase, entre ellas una de que veremos [más adelante](#Otra-forma-de-hacer-iterable-una-estructura-propia).

Empezamos generando datos con los que vamos a trabajar.

In [10]:
class Nodo:
    
    def __init__(self, valor, siguiente):
        self.valor = valor
        self.siguiente = siguiente
    
    def __repr__(self):
        return f"{self.valor}"


datos = Nodo(1, Nodo(2, Nodo(3, Nodo(4, Nodo(5, None)))))

Luego debemos crear la clase `Iterable` que implementará el método `__iter__`. 

In [11]:
class Iterable:
    
    def __init__(self, objeto):
        self.objeto = objeto
    
    def __iter__(self):
        return Iterador(self.objeto)

Ahora creamos la clase `Iterador` que será la encargada de iterar sobre el iterable. Esta debe implementar el método `__next__` que retornará los valores.

El método `__iter__` sólo debe retornar `self`. Es posible no implementarlo, pero no será posible obtener el iterador `iterador = iter(iterable)` y luego usar el iterador directamente en el `for`.

En general, los iteradores no se pueden reiniciar.

In [12]:
class Iterador:
    
    def __init__(self, iterable):
        self.iterable = iterable
    
    def __iter__(self): 
        return self
    
    def __next__(self):
        if self.iterable is None:
            # Así es como se levanta una excepción del tipo StopIteration
            # con el mensaje "Llegamos al final", veremos excepciones con 
            # más detalle en una clase futura.
            raise StopIteration("Llegamos al final")
        else:           
            valor = self.iterable
            self.iterable = self.iterable.siguiente
            return valor

In [13]:
iterable = Iterable(datos)
for i in iterable:
    print(i)

1
2
3
4
5


**Si el iterador no implementa `__iter__`, no se podrá guardar y usar directamente en un `for`**:

In [14]:
class IterableMalo:
    
    def __init__(self, objeto):
        self.objeto = objeto
    
    def __iter__(self):
        return IteradorMalo(self.objeto)

    
class IteradorMalo:
    
    def __init__(self, iterable):
        self.iterable = iterable
    
    def __next__(self):
        if self.iterable is None:
            raise StopIteration("Llegamos al final")
        else:           
            valor = self.iterable
            self.iterable = self.iterable.siguiente
            return valor

iterable = IterableMalo(datos)
iterador = iter(iterable)
for i in iterador:
    print(i)

TypeError: 'IteradorMalo' object is not iterable

Si paramos la iteración antes de recorrer todos los elementos, podemos continuar con el mismo iterador desde el punto en que lo dejamos:

In [15]:
iterable = Iterable(datos)
iterador = iter(iterable)
for i in iterador:
    print(i)
    if i.valor >= 3:
        break

1
2
3


In [16]:
for i in iterador:
    print(i)

4
5


Podemos volver a empezar **obteniendo otro iterador**. El código de abajo funciona, debido a que cuando se invoca otra vez la función `__iter__` del iterable, se retorna un **nuevo** iterador.

In [17]:
for i in iterable:
    print(i)

1
2
3
4
5


Cada iterador tiene su propia memoria que no depende del iterable. Para verlo, creamos dos iteradores:

In [18]:
iterador_1 = iter(iterable)
iterador_2 = iter(iterable)
type(iterador_1)

__main__.Iterador

Utilizamos el primero sólo tres veces

In [13]:
for i in iterador_1:
    print(i)
    if i.valor >= 3:
        break

1
2
3


Y el segundo iterador recorre la colección otra vez, independiente de lo recorrido por el primero.

In [14]:
for i in iterador_2:
    print(i)

1
2
3
4
5


Pero si volvemos a recorrer la estructura con el primer iterador (que sólo usamos tres veces), continuaremos desde donde lo dejamos:

In [15]:
for i in iterador_1:
    print(i)

4
5


Por último, una vez que el iterador agotó la estructura de datos no lo podemos utilizar de nuevo

In [16]:
for i in iterador_1:
    print(i)

## Generadores

Los **generadores** son un caso especial de los **iteradores**. Los generadores nos permiten iterar sobre secuencias de datos sin la necesidad de almacenarlos en alguna estructura especial, evitando el uso innecesario de memoria. 

Una vez que terminamos de iterar sobre un generador, el generador desaparece. Esto es muy útil cuando queremos realizar cálculos sobre secuencias de números que sólo nos sirven para ese cálculo en particular. La sintaxis para crear generadores es muy parecida a la comprensión de listas, sólo que en vez de paréntesis cuadrados `[` `]` usamos paréntesis normales `(` `)`.

Por ejemplo, creemos un generador para los números pares del 0 al 18:

In [1]:
generador_pares = (2 * i for i in range(10)) # Por el sólo hecho de usar paréntesis estamos creando un generador.

In [2]:
type(generador_pares)

generator

Ahora vamos a mostrar lo que nos entrega con un `for`. Esto es posible ya que **los generadores implementan `__iter__`** retornando `self`.

In [3]:
for i in generador_pares:
    print(i)

0
2
4
6
8
10
12
14
16
18


Nuevamente, como los generadores son un caso especial de iteradores, estos no se pueden usar otra vez, sino que tendremos que crear otro nuevo.

In [4]:
for i in generador_pares:
    print(i)

Recordar también que se puede usar `next`:

In [21]:
generador_pares = (2 * i for i in range(10))
print(next(generador_pares))
print(next(generador_pares))

0
2


Veamos cuánta memoria ocupa nuestro generador, _versus_ una lista que contiene todos los resultados. Para ello ocuparemos la función [`getsizeof`](https://docs.python.org/3/library/sys.html#sys.getsizeof).

In [22]:
from sys import getsizeof

In [23]:
generador_pares = (2 * i for i in range(10))
lista_pares = [2 * i for i in range(10)]

print("Bytes del generador:", getsizeof(generador_pares))
print("Bytes de la lista:", getsizeof(lista_pares))

Bytes del generador: 88
Bytes de la lista: 192


Esta diferencia es más grande si aumentamos la cantidad de resultados:

In [24]:
generador_pares = (2 * i for i in range(10 ** 6))
lista_pares = [2 * i for i in range(10 ** 6)]

print("Bytes del generador:", getsizeof(generador_pares))
print("Bytes de la lista:", getsizeof(lista_pares))

Bytes del generador: 88
Bytes de la lista: 8697464


## Funciones Generadoras

Las funciones en Python también tienen la posibilidad de funcionar como generadores, con la sentencia `yield`. El _statement_ `yield` es un análogo al `return`, con ciertas diferencias. Por un lado, `yield` se encarga de retornar el valor indicado, pero también se asegura que en la próxima llamada a la función, la ejecución parta desde donde se dejó antes. 

En otras palabras, trabajamos con una función que una vez que entrega un valor a través de `yield`, está cediendo el control sólo en forma temporal, asumiendo que pronto será utilizada nuevamente para generar más valores.

Creemos nuestra primera función generadora de números decrecientes:

In [25]:
def conteo_decreciente(n):
    print(f"Contando en forma decreciente desde {n}")
    while n > 0:
        yield n
        n -= 1

Vemos que cuando se llama a la función generadora, esta no ejecuta nada:

In [26]:
x = conteo_decreciente(10)

Esto se debe a que cuando se invoca la función generadora, esta retorna un **generador**. Luego, `x` es un generador de números desde el 10 hasta el 1.

In [27]:
type(x)

generator

Podemos usar el generador directamente en un `for`, ya que como vimos estos implementan `__iter__` devolviendo `self`.

In [28]:
for number in x:
    print(number)

Contando en forma decreciente desde 10
10
9
8
7
6
5
4
3
2
1


También se puede usar `next`:

In [29]:
x = conteo_decreciente(5)
print(next(x))

Contando en forma decreciente desde 5
5


### Ejemplos

Veamos un ejemplo de una función generadora de números de Fibonacci:

In [8]:
def fibonacci():
    a, b = 0, 1
    while True: # Notar que este generador nunca "se agota"
        yield b
        a, b = b, a + b

generador_fibonacci = fibonacci()

# Imprimimos los primeros 5 elementos
for i in range(5):
    print(next(generador_fibonacci))

1
1
2
3
5


También veamos un ejemplo de que las funciones generadoras pueden operar con otras colecciones, como listas:

In [31]:
def maximo_acumulativo(valores):
    """Retorna el máximo visto hasta ahora en una colección de valores."""
    max_ = float('-inf')
    for valor in valores:
        max_ = max(valor, max_)
        yield max_
        
lista = [1, 10, 14, 7, 9, 12, 19, 33]

for i in maximo_acumulativo(lista):
    print(i)

1
10
14
14
14
14
19
33


### Otra forma de hacer iterable una estructura propia

Si tenemos una estructura de datos propia, podemos usar una función generadora en `__iter__` en vez de crear nuestra propia clase iteradora. Para ilustrarlo usaremos el mismo ejemplo del principio:

In [9]:
class Nodo:
    
    def __init__(self, valor, siguiente):
        self.valor = valor
        self.siguiente = siguiente
    
    def __repr__(self):
        return f"{self.valor}"


datos = Nodo(1, Nodo(2, Nodo(3, Nodo(4, Nodo(5, None)))))

In [33]:
class Iterable:
    
    def __init__(self, objeto):
        self.objeto = objeto
    
    def __iter__(self):
        nodo_actual = self.objeto
        while nodo_actual:
            yield nodo_actual
            nodo_actual = nodo_actual.siguiente

In [34]:
iterable = Iterable(datos)

In [35]:
for x in iterable:
    print(x)

1
2
3
4
5


In [36]:
iterador = iter(iterable)
type(iterador)

generator

### También podemos interactuar con la función generadora enviando mensajes 

El método `send()` permite enviar un valor hacia el generador, lo que significa que la expresión `yield` lo recibirá. El valor enviado puede ser usado para asignarlo a otra variable, por ejemplo `v = yield self.value`.

Veamos un ejemplo de una función generadora que entrega números que se incrementan según lo que le es enviado mediante `send`:

In [36]:
def funcion_generadora_send():
    contador = 0
    while True:
        valor_recibido = yield contador
        print("Hemos recibido {}".format(valor_recibido))
        if valor_recibido is None:  # Consideraremos 0 si nos llega un None
            valor_recibido = 0
        print("Sumaremos {} a nuestro contador".format(valor_recibido))
        contador += valor_recibido  # Sumamos el valor recibido al contador que llevamos

In [37]:
generador_send = funcion_generadora_send()
#print(generador_send)

Lo primero en lo que nos tenemos que fijar es que debemos avanzar hasta `yield` antes de poder enviar valores. Es decir, la primera vez no podremos enviar nada, sólo usar `next`:

In [38]:
generador_send.send(5)

TypeError: can't send non-None value to a just-started generator

In [31]:
next(generador_send)

0

Ya hecho el primer `yield`, podemos enviarle valores a la función generadora:

In [25]:
generador_send.send(5)

Hemos recibido 5
Sumaremos 5 a nuestro contador


5

Hacer `next`, es equivalente a hacer `send` de `None`.

In [42]:
next(generador_send)

Hemos recibido None
Sumaremos 0 a nuestro contador


5

In [43]:
generador_send.send(10)

Hemos recibido 10
Sumaremos 10 a nuestro contador


15